### 저장된 Yolo 불러오기

In [1]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', path = './yolov5-master/exp3/weights/best.pt')
def net_yolo(img):
    h, w, c = img.shape
    pred_list = model(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    pred_list = pred_list.pandas().xyxy[0]
    result_list = []
    for cnt in range(len(pred_list)):
        x1 = int(pred_list.loc[cnt]['xmin'])
        y1 = int(pred_list.loc[cnt]['ymin'])
        x2 = int(pred_list.loc[cnt]['xmax'])
        y2 = int(pred_list.loc[cnt]['ymax'])
        x1_nor = max(1, int(pred_list.loc[cnt]['xmin'])) / w
        y1_nor = max(1, int(pred_list.loc[cnt]['ymin'])) / h
        x2_nor = max(1, int(pred_list.loc[cnt]['xmax'])) / w
        y2_nor = max(1, int(pred_list.loc[cnt]['ymax'])) / h
        conf = round(float(pred_list.loc[cnt]['confidence']), 3)
        pred = {'bbox' : [x1, y1, x2, y2], 'conf' : conf, 'bbox_nor' : [x1_nor, y1_nor, x2_nor, y2_nor]}
        result_list.append(pred)
    if len(result_list) > 0:
        result_list.sort(key = lambda x:x['conf'], reverse = True)
        result = result_list[0]                    
        crop_img = img[result['bbox'][1]:result['bbox'][3], result['bbox'][0]:result['bbox'][2]]
        return result['bbox_nor'], crop_img
    else:
        return [], []

Using cache found in C:\Users\nyan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-15 Python-3.8.15 torch-1.10.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 


### 저장된 Resnet 불러오기

In [2]:
def seed(seed = 1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
if torch.cuda.is_available() == True:
    device = 'cuda:0'
    print('현재 가상환경 GPU 사용 가능상태')
else:
    device = 'cpu'
    print('GPU 사용 불가능 상태')

현재 가상환경 GPU 사용 가능상태


In [3]:
from torchvision import models
import torch.nn as nn
from torchsummary import summary as Summary

seed()
model_resnet = models.resnet152(pretrained = True)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 4)
model_resnet.load_state_dict(torch.load('./best_model_cnn.pth'))
net_cnn = model_resnet.to(device)
net_cnn.eval()
Summary(net_cnn.to(device), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

### MediaPipe 모델 불러오기

In [4]:
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands =  mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)
def net_mediapipe(img):
    results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if not results.multi_hand_landmarks: return []
    xy_list = []
    for one_hand in results.multi_hand_landmarks:
        for xy in one_hand.landmark:
            xy_list.append(xy.x)
            xy_list.append(xy.y)        
    return xy_list # 42개

### LSTM을 위한 데이터 만들기(Yolo + Resnet + MediaPipe)

In [5]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Resize, Normalize
import numpy as np
import os
import cv2
import natsort
from tqdm import tqdm

seed()
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
data_transforms = transforms.Compose([ToTensor(), Resize((224,224)), Normalize(mean, std)])
class dataset_cnn(Dataset):
    def __init__(self, img, label):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = data_transforms(img)
        self.img = [img]
        self.label = [label]
    def __getitem__(self, index):
        data = self.img[index]
        label = self.label[index]
        return data, label
    def __len__(self):
        return len(self.label)
    
def list_augmentation(input_list, goal_len):
    need_aug = goal_len - len(input_list)
    aug_tempo = need_aug / len(input_list)
    full_list, cnt = [], 0
    for _input in input_list:
        cnt += aug_tempo
        full_list.append(_input)
        while True:
            if cnt < 1: break
            full_list.append(_input)
            cnt -= 1
    while True:
        if len(full_list) == goal_len: break
        if len(full_list) > goal_len: del full_list[-1]
        else: full_list.append(full_list[-1])    
    return full_list

In [6]:
last_mp_result = []
for i in range(42):
    last_mp_result.append(0.0)
train_video_path = './train_video'
folder_list = os.listdir(train_video_path)
folder_list = natsort.natsorted(folder_list)
dataset_dict_list = []
for folder in folder_list:
    video_list = os.listdir('{}/{}'.format(train_video_path, folder))
    video_list = natsort.natsorted(video_list)
    for video_name in tqdm(video_list):
        logit_bbox_mp_list = []
        cap = cv2.VideoCapture('{}/{}/{}'.format(train_video_path, folder, video_name))
        if cap.isOpened():
            while True:
                ret, img = cap.read()
                if ret == True:
                    bbox, crop_img = net_yolo(img)
                    if len(crop_img) == 0: continue
                    dataset = dataset_cnn(crop_img, int(folder))
                    dataset = DataLoader(dataset)
                    for data, label in dataset:
                        data = data.to(device)
                        with torch.no_grad():
                            cnn_result = net_cnn(data)
                        cnn_logits = []
                        for logit in cnn_result[0]:
                            cnn_logits.append(logit.item())
                        mp_result = net_mediapipe(img)
                        if len(mp_result) == 0: mp_result = last_mp_result + []
                        last_mp_result = mp_result + []
                        logit_bbox_mp = cnn_logits + bbox + last_mp_result
                        logit_bbox_mp_list.append(logit_bbox_mp)
                else: break
        logit_bbox_mp_list = list_augmentation(logit_bbox_mp_list, 30)
        dataset_dict_list.append({'key':int(folder), 'value':logit_bbox_mp_list})

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [06:48<00:00,  3.58s/it]


### LSTM 설계 후 학습

In [7]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from torch.optim import Adam
import gc
from torch.cuda import memory_allocated, empty_cache
from IPython.display import clear_output

class dataset_lstm(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list :
            self.y.append(dic['key'])
            self.X.append(dic['value'])
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))
    def __len__(self):
        return len(self.X)
    
split_ratio = [0.9, 0.1]
train_len = int(len(dataset_dict_list) * split_ratio[0])
val_len = len(dataset_dict_list) - train_len
print('train: {}, val: {}'.format(train_len, val_len))

train_dataset = dataset_lstm(dataset_dict_list)
train_data, valid_data = random_split(train_dataset, [train_len, val_len])
train_loader = DataLoader(train_data, batch_size=32)
val_loader = DataLoader(valid_data, batch_size=32)

train: 549, val: 61


In [8]:
class skeleton_LSTM(nn.Module) :
    def __init__(self):
        super(skeleton_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=50, hidden_size=128, num_layers=1, batch_first=True)
        self.batch1 = nn.BatchNorm1d(30)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=1, batch_first=True)
        self.batch2 = nn.BatchNorm1d(30)
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=1, batch_first=True)
        self.batch3 = nn.BatchNorm1d(30)
        self.lstm4 = nn.LSTM(input_size=512, hidden_size=1024, num_layers=1, batch_first=True)
        self.batch4 = nn.BatchNorm1d(30)
        self.dropout1 = nn.Dropout(0.4)
        self.lstm5 = nn.LSTM(input_size=1024, hidden_size=512, num_layers=1, batch_first=True)
        self.batch5 = nn.BatchNorm1d(30)
        self.lstm6 = nn.LSTM(input_size=512, hidden_size=256, num_layers=1, batch_first=True)
        self.batch6 = nn.BatchNorm1d(30)
        self.lstm7 = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True)
        self.batch7 = nn.BatchNorm1d(30)
        self.lstm8 = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, batch_first=True)
        self.batch8 = nn.BatchNorm1d(30)
        self.dropout2 = nn.Dropout(0.4)
        self.lstm9 = nn.LSTM(input_size=64, hidden_size=32, num_layers=1, batch_first=True)
        self.batch9 = nn.BatchNorm1d(30)
        self.fc = nn.Linear(32,5)
    def forward(self, x) :
        x, _ = self.lstm1(x)
        x = self.batch1(x)
        x, _ = self.lstm2(x)
        x = self.batch2(x)
        x, _ = self.lstm3(x)
        x = self.batch3(x)
        x, _ = self.lstm4(x)
        x = self.batch4(x)
        x = self.dropout1(x)
        x, _ = self.lstm5(x)
        x = self.batch5(x)
        x, _ = self.lstm6(x)
        x = self.batch6(x)
        x, _ = self.lstm7(x)
        x = self.batch7(x)
        x, _ = self.lstm8(x)
        x = self.batch8(x)
        x = self.dropout2(x)
        x, _ = self.lstm9(x)
        x = self.batch9(x)
        x = self.fc(x[:,-1,:])
        return x
    
def init_model():
    plt.rc('font', size = 10)
    global net_lstm, loss_fn, optim
    net_lstm = skeleton_LSTM().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = Adam(net_lstm.parameters(), lr=0.0001)
    
def init_epoch():
    global epoch_cnt
    epoch_cnt = 0
    
def init_log():
    plt.rc('font', size = 10)
    global log_stack, iter_log, tloss_log, tacc_log, vloss_log, vacc_log, time_log
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    time_log, log_stack = [], []
    
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()
    
def epoch(data_loader, mode = 'train'):
    global epoch_cnt
    iter_loss, iter_acc, last_grad_performed = [], [], False
    for _data, _label in data_loader:
        data, label = _data.to(device), _label.type(torch.LongTensor).to(device)
        if mode == 'train':
            net_lstm.train()
        else:
            net_lstm.eval()
        result = net_lstm(data)
        _, out = torch.max(result, 1)
        loss = loss_fn(result, label)
        iter_loss.append(loss.item())
        if mode == 'train':
            optim.zero_grad()
            loss.backward()
            optim.step()
            last_grad_performed = True
        acc_partial = (out == label).float().sum()
        acc_partial = acc_partial / len(label) 
        iter_acc.append(acc_partial.item())
    if last_grad_performed:
        epoch_cnt += 1
    clear_memory()
    return np.average(iter_loss), np.average(iter_acc)

def epoch_not_finished():
    return epoch_cnt < maximum_epoch

def record_train_log(_tloss, _tacc, _time):
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)
    
def record_valid_log(_vloss, _vacc):
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)
    
def last(log_list):
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1
    
def print_log(memo):
    train_loss = round(float(last(tloss_log)), 6)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 6)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)
    log_str = 'Epoch: {:3} | T_Loss {:5} | T_acc {:5} | V_Loss {:5} | V_acc. {:5} | \
🕒 {:5} {}'.format(last(iter_log), train_loss, train_acc, val_loss, val_acc, time_spent, memo)
    log_stack.append(log_str)
    hist_fig, loss_axis = plt.subplots(figsize=(10, 3), dpi=99) # 그래프 사이즈 설정
    hist_fig.patch.set_facecolor('white') # 그래프 배경색 설정
    loss_t_line = plt.plot(iter_log, tloss_log, label='Train Loss', color='red', marker='o')
    loss_v_line = plt.plot(iter_log, vloss_log, label='Valid Loss', color='blue', marker='s')
    loss_axis.set_xlabel('epoch')
    loss_axis.set_ylabel('loss')
    acc_axis = loss_axis.twinx()
    acc_t_line = acc_axis.plot(iter_log, tacc_log, label='Train Acc.', color='red', marker='+')
    acc_v_line = acc_axis.plot(iter_log, vacc_log, label='Valid Acc.', color='blue', marker='x')
    acc_axis.set_ylabel('accuracy')
    hist_lines = loss_t_line + loss_v_line + acc_t_line + acc_v_line # 위에서 선언한 plt정보들 통합
    loss_axis.legend(hist_lines, [l.get_label() for l in hist_lines]) # 순서대로 그려주기
    loss_axis.grid() # 격자 설정
    plt.title('Learning history until epoch {}'.format(last(iter_log)))
    plt.draw()
    clear_output(wait=True)
    plt.show()
    for idx in reversed(range(len(log_stack))): # 반대로 sort 시켜서 출력
        print(log_stack[idx])

In [9]:
import time

init_model()
init_epoch()
init_log()
maximum_epoch = 300
min_loss = 1
memo = ''
while epoch_not_finished():
    start_time = time.time()
    tloss, tacc = epoch(train_loader, mode = 'train')
    end_time = time.time()
    time_taken = end_time - start_time
    record_train_log(tloss, tacc, time_taken)
    with torch.no_grad():
        vloss, vacc = epoch(val_loader, mode = 'val')
        if vloss < min_loss:
            print('최소 Loss 달성. 모델 저장: {}'.format(epoch_cnt))
            min_loss = vloss
            torch.save(net_lstm.state_dict(), './model_lstm_yolo_cnn_mediapipe.pth')
            memo = '<-- 저장 포인트'
        else: memo = ''
        record_valid_log(vloss, vacc)
    print_log(memo)
print('\n Training completed!')

Epoch: 300 | T_Loss 0.000714 | T_acc   1.0 | V_Loss 0.295248 | V_acc. 0.967 | 🕒 1.484 
Epoch: 299 | T_Loss 0.000728 | T_acc   1.0 | V_Loss 0.294946 | V_acc. 0.967 | 🕒 1.469 
Epoch: 298 | T_Loss 0.000727 | T_acc   1.0 | V_Loss 0.293537 | V_acc. 0.967 | 🕒   1.5 
Epoch: 297 | T_Loss 0.000732 | T_acc   1.0 | V_Loss 0.293387 | V_acc. 0.967 | 🕒 1.516 
Epoch: 296 | T_Loss 0.000743 | T_acc   1.0 | V_Loss 0.293461 | V_acc. 0.967 | 🕒 1.484 
Epoch: 295 | T_Loss 0.000752 | T_acc   1.0 | V_Loss 0.293179 | V_acc. 0.967 | 🕒 1.469 
Epoch: 294 | T_Loss 0.000772 | T_acc   1.0 | V_Loss 0.292393 | V_acc. 0.967 | 🕒 1.484 
Epoch: 293 | T_Loss 0.000771 | T_acc   1.0 | V_Loss 0.291826 | V_acc. 0.967 | 🕒 1.547 
Epoch: 292 | T_Loss 0.000773 | T_acc   1.0 | V_Loss 0.291787 | V_acc. 0.967 | 🕒 1.547 
Epoch: 291 | T_Loss 0.000787 | T_acc   1.0 | V_Loss 0.291536 | V_acc. 0.967 | 🕒 1.469 
Epoch: 290 | T_Loss 0.000793 | T_acc   1.0 | V_Loss 0.290762 | V_acc. 0.967 | 🕒 1.453 
Epoch: 289 | T_Loss 0.000805 | T_acc   1.0 

### Validation 없이 정해진 epoch 만큼 무대포 Train

In [10]:
import random
import copy
clear_memory()

only_train = copy.deepcopy(dataset_dict_list)
random.shuffle(only_train)
train_dataset = dataset_lstm(only_train)
train_loader = DataLoader(train_dataset, batch_size=32)
init_model()
maximum_epoch = 200
for i in tqdm(range(maximum_epoch)):
    tloss, tacc = epoch(train_loader, mode = 'train')
    print('Train Loss: {} | Train Acc.: {}'.format(tloss, tacc))
torch.save(net_lstm.state_dict(), './model_lstm_yolo_cnn_mediapipe_wholetrain.pth')
print('\n Training completed!')

  0%|▍                                                                                 | 1/200 [00:01<05:50,  1.76s/it]

Train Loss: 0.6772672839462757 | Train Acc.: 0.7671875


  1%|▊                                                                                 | 2/200 [00:03<05:30,  1.67s/it]

Train Loss: 0.42971356213092804 | Train Acc.: 0.8875


  2%|█▏                                                                                | 3/200 [00:05<05:26,  1.66s/it]

...

Train Loss: 0.001659173733787611 | Train Acc.: 1.0


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [05:24<00:01,  1.61s/it]

Train Loss: 0.001628487603738904 | Train Acc.: 1.0


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [05:26<00:00,  1.63s/it]

Train Loss: 0.0015903397696092724 | Train Acc.: 1.0

 Training completed!
